In [5]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [6]:
file_path = "./jsonl/batch_qiX2iulpCqqbDZqSC29hLPl2_output.jsonl"
gt_data = pd.read_json(f"./images/configurations.json")

In [7]:
# Dictionary to map custom_id to message content
custom_id_to_content = {}

# Open the file and read line by line
with open(file_path, "r") as file:
    for line in file:
        json_obj = json.loads(line)
        # Extract custom_id and message content
        custom_id = json_obj.get("custom_id")
        message_content = json_obj["response"]["body"]["choices"][0]["message"][
            "content"
        ]
        # Map custom_id to message content
        if custom_id:
            custom_id_to_content[custom_id.replace("uid__", "")] = message_content

# Print the dictionary to verify
print(len(custom_id_to_content))

120


In [8]:
# drop squares column
gt_data = gt_data.drop(columns=["squares"])

In [9]:
# ad a new column to gt_data with content of  custom_id_to_content (match key with image_path)
gt_data["message"] = gt_data["image_name"].map(custom_id_to_content)
gt_data

,image_name,depth,center,initial_size,reduction_factor,line_thickness,padding,message
0,nested_squares_depth_2_image_1_thickness_2.png,2,"[-2.2395532434656262, -2.245365217594155]",17.574157,0.75,2,0.75,The image contains two squares: one inner squa...
1,nested_squares_depth_2_image_1_thickness_3.png,2,"[-2.2395532434656262, -2.245365217594155]",17.574157,0.75,3,0.75,The image contains 2 squares: one smaller squa...
2,nested_squares_depth_2_image_1_thickness_4.png,2,"[-2.2395532434656262, -2.245365217594155]",17.574157,0.75,4,0.75,"In the image, there are two nested squares. \n..."
3,nested_squares_depth_2_image_2_thickness_2.png,2,"[4.522003363881829, 1.575293928741691]",12.158126,0.75,2,0.75,There are two squares in the image: one smalle...
4,nested_squares_depth_2_image_2_thickness_3.png,2,"[4.522003363881829, 1.575293928741691]",12.158126,0.75,3,0.75,The image contains 2 squares; one small square...
...,...,...,...,...,...,...,...,...
115,nested_squares_depth_5_image_9_thickness_3.png,5,"[4.67396043765117, -3.083874562085932]",10.089593,0.75,3,0.75,There are a total of 6 squares in the image.
116,nested_squares_depth_5_image_9_thickness_4.png,5,"[4.67396043765117, -3.083874562085932]",10.089593,0.75,4,0.75,There are 6 nested squares in the image.
117,nested_squares_depth_5_image_10_thickness_2.png,5,"[1.960921482573098, 0.529244414005689]",8.411051,0.75,2,0.75,There are 5 squares in the image.
118,nested_squares_depth_5_image_10_thickness_3.png,5,"[1.960921482573098, 0.529244414005689]",8.411051,0.75,3,0.75,The image contains a series of concentric squa...


In [10]:
import re


# Function to extract numbers from text, including zero represented as text
def extract_number(text):
    # Check for phrases that imply zero intersections
    if (
        "no intersections" in text
        or "zero line intersections" in text
        or "no line" in text
    ):
        return 0
    # First try to find digits
    match = re.search(r"\b\d+\b", text)
    if match:
        return int(match.group())
    # If no digits, look for text numbers (one, two, three, etc.)
    text_to_num = {"one": 1, "two": 2, "three": 3, "four": 4, "five": 5}
    for word, num in text_to_num.items():
        if word in text:
            return num
    return None


# Apply the function to the 'message' column
gt_data["extracted_number"] = gt_data["message"].apply(extract_number)
gt_data[["image_name", "message", "extracted_number"]]

,image_name,message,extracted_number
0,nested_squares_depth_2_image_1_thickness_2.png,The image contains two squares: one inner squa...,1
1,nested_squares_depth_2_image_1_thickness_3.png,The image contains 2 squares: one smaller squa...,2
2,nested_squares_depth_2_image_1_thickness_4.png,"In the image, there are two nested squares. \n...",2
3,nested_squares_depth_2_image_2_thickness_2.png,There are two squares in the image: one smalle...,1
4,nested_squares_depth_2_image_2_thickness_3.png,The image contains 2 squares; one small square...,2
...,...,...,...
115,nested_squares_depth_5_image_9_thickness_3.png,There are a total of 6 squares in the image.,6
116,nested_squares_depth_5_image_9_thickness_4.png,There are 6 nested squares in the image.,6
117,nested_squares_depth_5_image_10_thickness_2.png,There are 5 squares in the image.,5
118,nested_squares_depth_5_image_10_thickness_3.png,The image contains a series of concentric squa...,6


In [11]:
gt_data

,image_name,depth,center,initial_size,reduction_factor,line_thickness,padding,message,extracted_number
0,nested_squares_depth_2_image_1_thickness_2.png,2,"[-2.2395532434656262, -2.245365217594155]",17.574157,0.75,2,0.75,The image contains two squares: one inner squa...,1
1,nested_squares_depth_2_image_1_thickness_3.png,2,"[-2.2395532434656262, -2.245365217594155]",17.574157,0.75,3,0.75,The image contains 2 squares: one smaller squa...,2
2,nested_squares_depth_2_image_1_thickness_4.png,2,"[-2.2395532434656262, -2.245365217594155]",17.574157,0.75,4,0.75,"In the image, there are two nested squares. \n...",2
3,nested_squares_depth_2_image_2_thickness_2.png,2,"[4.522003363881829, 1.575293928741691]",12.158126,0.75,2,0.75,There are two squares in the image: one smalle...,1
4,nested_squares_depth_2_image_2_thickness_3.png,2,"[4.522003363881829, 1.575293928741691]",12.158126,0.75,3,0.75,The image contains 2 squares; one small square...,2
...,...,...,...,...,...,...,...,...,...
115,nested_squares_depth_5_image_9_thickness_3.png,5,"[4.67396043765117, -3.083874562085932]",10.089593,0.75,3,0.75,There are a total of 6 squares in the image.,6
116,nested_squares_depth_5_image_9_thickness_4.png,5,"[4.67396043765117, -3.083874562085932]",10.089593,0.75,4,0.75,There are 6 nested squares in the image.,6
117,nested_squares_depth_5_image_10_thickness_2.png,5,"[1.960921482573098, 0.529244414005689]",8.411051,0.75,2,0.75,There are 5 squares in the image.,5
118,nested_squares_depth_5_image_10_thickness_3.png,5,"[1.960921482573098, 0.529244414005689]",8.411051,0.75,3,0.75,The image contains a series of concentric squa...,6


In [12]:
# count none or na in gt_data extracted_number
gt_data["extracted_number"].isna().sum()
# show their value in the message (same row)
gt_data[gt_data["extracted_number"].isna()]["message"]

Series([], Name: message, dtype: object)

In [13]:
# Cast depth to integer
gt_data["depth"] = gt_data["depth"].astype(int)

# Cast extracted_number to integer (it should already be integer from the extraction function, but just to ensure consistency)
gt_data["extracted_number"] = gt_data["extracted_number"].astype(int)

# Compare depth and extracted_number to check if they are the same
gt_data["is_correct"] = gt_data["depth"] == gt_data["extracted_number"]

# Display the DataFrame with the new column
gt_data[["image_name", "depth", "extracted_number", "is_correct"]]

,image_name,depth,extracted_number,is_correct
0,nested_squares_depth_2_image_1_thickness_2.png,2,1,False
1,nested_squares_depth_2_image_1_thickness_3.png,2,2,True
2,nested_squares_depth_2_image_1_thickness_4.png,2,2,True
3,nested_squares_depth_2_image_2_thickness_2.png,2,1,False
4,nested_squares_depth_2_image_2_thickness_3.png,2,2,True
...,...,...,...,...
115,nested_squares_depth_5_image_9_thickness_3.png,5,6,False
116,nested_squares_depth_5_image_9_thickness_4.png,5,6,False
117,nested_squares_depth_5_image_10_thickness_2.png,5,5,True
118,nested_squares_depth_5_image_10_thickness_3.png,5,6,False


In [14]:
gt_data["is_correct"].value_counts()

is_correct
False    62
True     58
Name: count, dtype: int64

In [15]:
100 * np.mean(gt_data["is_correct"])

48.333333333333336

In [16]:
# Assuming 'cleaned_data' DataFrame has columns 'line_thickness', 'num_intersections', and 'extracted_number'

# Convert line_thickness to an appropriate numeric type if necessary
gt_data["line_thickness"] = gt_data["line_thickness"].astype(int)

# Calculate accuracy for each thickness
accuracy_by_thickness = gt_data.groupby("line_thickness").apply(
    lambda df: (df["is_correct"]).mean()
)

accuracy_by_thickness = 100 * accuracy_by_thickness.round(2)
accuracy_by_thickness

/var/folders/y1/b5wjnkp96_b7cvzncx0zsry40000gn/T/ipykernel_20194/388290072.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  accuracy_by_thickness = gt_data.groupby("line_thickness").apply(


line_thickness
2    52.0
3    45.0
4    48.0
dtype: float64

In [17]:
# Convert line_thickness to an appropriate numeric type if necessary
gt_data["depth"] = gt_data["depth"].astype(int)

# Calculate accuracy for each thickness
accuracy_by_thickness = gt_data.groupby("depth").apply(
    lambda df: (df["is_correct"]).mean()
)

accuracy_by_thickness = 100 * accuracy_by_thickness.round(2)
accuracy_by_thickness

/var/folders/y1/b5wjnkp96_b7cvzncx0zsry40000gn/T/ipykernel_20194/210657226.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  accuracy_by_thickness = gt_data.groupby("depth").apply(


depth
2    77.0
3    40.0
4    20.0
5    57.0
dtype: float64

# Export

In [18]:
gt_data["Model"] = ["gpt-4o"] * len(gt_data)
# rename message to model_output_raw
gt_data.rename(columns={"message": "model_output_raw"}, inplace=True)
gt_data.to_pickle("./data/gpt-4o.pkl")

In [19]:
gt_data

,image_name,depth,center,initial_size,reduction_factor,line_thickness,padding,model_output_raw,extracted_number,is_correct,Model
0,nested_squares_depth_2_image_1_thickness_2.png,2,"[-2.2395532434656262, -2.245365217594155]",17.574157,0.75,2,0.75,The image contains two squares: one inner squa...,1,False,gpt-4o
1,nested_squares_depth_2_image_1_thickness_3.png,2,"[-2.2395532434656262, -2.245365217594155]",17.574157,0.75,3,0.75,The image contains 2 squares: one smaller squa...,2,True,gpt-4o
2,nested_squares_depth_2_image_1_thickness_4.png,2,"[-2.2395532434656262, -2.245365217594155]",17.574157,0.75,4,0.75,"In the image, there are two nested squares. \n...",2,True,gpt-4o
3,nested_squares_depth_2_image_2_thickness_2.png,2,"[4.522003363881829, 1.575293928741691]",12.158126,0.75,2,0.75,There are two squares in the image: one smalle...,1,False,gpt-4o
4,nested_squares_depth_2_image_2_thickness_3.png,2,"[4.522003363881829, 1.575293928741691]",12.158126,0.75,3,0.75,The image contains 2 squares; one small square...,2,True,gpt-4o
...,...,...,...,...,...,...,...,...,...,...,...
115,nested_squares_depth_5_image_9_thickness_3.png,5,"[4.67396043765117, -3.083874562085932]",10.089593,0.75,3,0.75,There are a total of 6 squares in the image.,6,False,gpt-4o
116,nested_squares_depth_5_image_9_thickness_4.png,5,"[4.67396043765117, -3.083874562085932]",10.089593,0.75,4,0.75,There are 6 nested squares in the image.,6,False,gpt-4o
117,nested_squares_depth_5_image_10_thickness_2.png,5,"[1.960921482573098, 0.529244414005689]",8.411051,0.75,2,0.75,There are 5 squares in the image.,5,True,gpt-4o
118,nested_squares_depth_5_image_10_thickness_3.png,5,"[1.960921482573098, 0.529244414005689]",8.411051,0.75,3,0.75,The image contains a series of concentric squa...,6,False,gpt-4o
